# Getting started with Plexiglass

## Model construction

For simplicity, we are going to use a pretrained resnet-18 model with MNIST test dataset


In [2]:
# import torch-related libraries
import torch
import torchvision
import torch.nn as nn
from torchvision import models

# import plexiglass
import plexiglass
from plexiglass.adversarial import FGSM, test_robustness

use_cuda=True

# define device
device = torch.device("cuda" if use_cuda else "cpu")

# define model
model = models.resnet18(pretrained=True).to(device)

# modify model for MNIST
from torchvision.models.resnet import ResNet, BasicBlock, Bottleneck

class MNISTResNet(ResNet):
    def __init__(self):
        super(MNISTResNet, self).__init__(BasicBlock, [2, 2, 2, 2], num_classes=10) # Based on ResNet18
        # super(MNISTResNet, self).__init__(BasicBlock, [3, 4, 6, 3], num_classes=10) # Based on ResNet34
        # super(MNISTResNet, self).__init__(Bottleneck, [3, 4, 6, 3], num_classes=10) # Based on ResNet50
        self.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=1, padding=3,bias=False)

model = MNISTResNet()
print(model)

MNISTResNet(
  (conv1): Conv2d(1, 64, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=Tru

Now load the dataset from PyTorch's default loader for MNIST

In [3]:
loader = torch.utils.data.DataLoader(
  torchvision.datasets.MNIST('/files/', train=False, download=True,
                             transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Normalize(
                                 (0.1307,), (0.3081,))
                             ])),
  batch_size=10, shuffle=False)

We can then use the plexiglass's test_robustness along with FGSM attack to test the model's robustness

In [6]:
# first define the attack
model = model.eval().to(device)
attack = FGSM(model=model, loss=nn.CrossEntropyLoss(), device=device)

# test robustness
test_robustness(attack=attack, dataloader=loader, device=device, eps=[0.07, 0.1, 0.5])

Accuracy: 99.1
Epsilon: 0.07 Accuracy: 99.0
Epsilon: 0.1 Accuracy: 98.2
Epsilon: 0.5 Accuracy: 93.5
